In [38]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# Function 1
# Load CSV into dictionary
data = pd.read_csv('test.csv', header=0, keep_default_na=False)
data = data.dropna()
print(data.shape)
print(list(data.columns))

(30, 19)
['room_id', 'survey_id', 'host_id', 'room_type', 'country', 'city', 'borough', 'neighborhood', 'reviews', 'overall_satisfaction', 'accommodates', 'bedrooms', 'bathrooms', 'price', 'minstay', 'last_modified', 'latitude', 'longitude', 'location']


In [39]:
data.head()

,room_id,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
0,11492254,1280,60361566,Shared room,,Singapore,,MK10,12,4.5,1,1,,29,,2017-05-17 9:09:06,1.350847,103.754883,0101000020E61000009C53C90050F0594038656EBE119D...
1,9974607,1280,51130590,Shared room,,Singapore,,TS16,24,4.0,8,1,,18,,2017-05-17 9:09:06,1.305768,103.855832,0101000020E6100000B3B794F3C5F65940FE9C82FC6CE4...
2,14489733,1280,87731750,Shared room,,Singapore,,TS17,3,4.5,12,1,,14,,2017-05-17 9:09:06,1.314830,103.859609,0101000020E61000003A9677D503F759407DCB9C2E8B09...
3,8654132,1280,45368183,Shared room,,Singapore,,TS06,19,4.5,8,1,,37,,2017-05-17 9:09:46,1.282850,103.844504,0101000020E6100000D655815A0CF6594011C7BAB88D86...
4,15089063,1280,11441748,Shared room,,Singapore,,MK26,8,5.0,2,1,,36,,2017-05-17 9:09:46,1.308946,103.902558,0101000020E6100000902FA182C3F9594025AE635C71F1...


In [44]:
# Function 2
# 1. What are the top 10 priciest rooms in Singapore? List the room_id as well as price
df2 = pd.DataFrame(data, columns=['room_id','price'])
df2.nlargest(10, 'price')

,room_id,price
6,12992216,567
14,11380394,482
10,14389000,441
9,2615452,410
8,11193032,408
13,10080333,361
7,9239776,345
12,13837153,322
11,13903438,315
22,11380161,295


In [83]:
# 2. Which hosts in Singapore have got more than 30 rooms? How many of such hosts do we have? 
# (Note: Each host can have more than one property listed, which in turn has varying number of rooms)
host_rooms = pd.DataFrame(data, columns=['host_id','room_id'])
hostGroup = host_rooms.groupby(['host_id'])
pd.DataFrame(hostGroup.size())

,0
host_id,
1128954,2
1439258,1
3926646,1
8492007,4
8527085,2
11114502,1
11441748,1
31464513,1
32184450,1


In [84]:
# 3. What are the different room types, and how many of each room types are there in Singapore?
pd.DataFrame(data.room_type.unique())
room_types = pd.DataFrame(data, columns=['room_type','room_id'])
roomTypeGroups = room_types.groupby(['room_type'])
pd.DataFrame(roomTypeGroups.size())

,0
room_type,
Entire home/apt,20
Private room,1
Shared room,9


In [89]:
# 4.What are the top 10 most reviewed rooms?
rooms = pd.DataFrame(data, columns=['room_id','reviews'])
pd.DataFrame(rooms).nlargest(10, 'reviews')

,room_id,reviews
15,10030323,42
9,2615452,33
29,9735434,25
1,9974607,24
16,10040419,23
21,5095636,23
26,9718911,23
18,7903662,22
3,8654132,19
11,13903438,19


In [90]:
# 5.What are the top 10 rooms with the highest overall satisfaction?
roomsOverallSatisf = pd.DataFrame(data, columns=['room_id','overall_satisfaction'])
pd.DataFrame(roomsOverallSatisf).nlargest(10, 'overall_satisfaction')

,room_id,overall_satisfaction
4,15089063,5.0
6,12992216,5.0
17,13129971,5.0
0,11492254,4.5
2,14489733,4.5
3,8654132,4.5
5,14489603,4.5
7,9239776,4.5
8,11193032,4.5
10,14389000,4.5


In [ ]:
# Function 3: Satisfaction Map